In [1]:
SSH_CONFIG_DIR = "/content/drive/MyDrive/projects/qmlcourse/config"

CONDA_ENVIRONMENT = "qmlcourse"
CONDA_ENV_PATH = f"/content/anaconda3/envs/{CONDA_ENVIRONMENT}"

# Install Anaconda

In [2]:
%%bash
wget -q https://repo.anaconda.com/archive/Anaconda3-2021.05-Linux-x86_64.sh
/bin/bash ./Anaconda3-2021.05-Linux-x86_64.sh -b -p /content/anaconda3 >/content/01_install_anaconda.log
rm -rf /content/Anaconda3-2021.05-Linux-x86_64.sh /content/sample_data
/content/anaconda3/bin/conda update -n base -c defaults conda >/content/02_conda_update.log

## Install Python 3.8 + Poetry

In [3]:
%%bash -s "$CONDA_ENVIRONMENT"
/content/anaconda3/bin/conda create -n ${1} python=3.8 poetry --yes >/content/03_conda_create.log

# Clone Project

In [4]:
! git clone https://github.com/SemyonSinchenko/qmlcourse

Cloning into 'qmlcourse'...
remote: Enumerating objects: 43283, done.
remote: Counting objects: 100% (3214/3214), done.
remote: Compressing objects: 100% (1206/1206), done.
remote: Total 43283 (delta 2226), reused 2795 (delta 1886), pack-reused 40069
Receiving objects: 100% (43283/43283), 445.10 MiB | 18.41 MiB/s, done.
Resolving deltas: 100% (30550/30550), done.


# Install Poetry Environment

In [5]:
%%bash -s "$CONDA_ENV_PATH"
cd qmlcourse && PYTHONPATH=${1}/lib/python3.8/site-packages ${1}/bin/poetry install >/content/04_poetry_install.log

# Install, Configure and Start SSH

In [6]:
%%bash -s "$SSH_CONFIG_DIR"
apt-get install -y ssh tmux vim >/content/05_install_ssh.log

# Configurate SSH
mkdir -p /root/.ssh
cp ${1}/config ${HOME}/.ssh  
cp ${1}/authorized_keys ${HOME}/.ssh  

# Restart SSH Service
service ssh restart

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


# Run Jupyter & SSH Port Forwarding (via ngrok)

In [8]:
%%bash -s "$CONDA_ENV_PATH"

# "Fix" google.colab extensions :)
rm -f /etc/ipython/ipython_config.py /etc/jupyter/jupyter_notebook_config.py

# Run Jupyter Notebook
cd qmlcourse && PYTHONPATH=${1}/lib/python3.8/site-packages \
  nohup ${1}/bin/poetry run jupyter notebook \
  --no-browser --notebook-dir=/content --allow-root --ip=localhost --port=8888 \
  --NotebookApp.token='' \
  --NotebookApp.nbserver_extensions="{'google.colab._serverextension':False}" >/content/06_run_jupyter.log 2>&1 &

# Set up ngrok SSH tunnel
ssh -R 0:localhost:22 tunnel.us.ngrok.com tcp 22


Process is interrupted.
